In [1]:
import warnings
warnings.filterwarnings('ignore')
import os 
import pandas as pd

In [2]:
master_data = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/data_files/cbd_master_deduped.csv',
                          dtype = object)
batch_data = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/data_files/sample_edc.csv',
                          dtype = object)

#### Input samples:

In [4]:
master_data.head(2)

,Unnamed: 0,corporation_number,business_number,business_name,alternate_name,category,subcategory,description,location_address_source,location_street_address,...,url,modified_at,retrieved_at,source_url,source_title,cleaned_business_name,cleaned_location_address_source,clusts,name_match_score,address_match_score
0,0,NaN,NaN,H2O INNOVATION INC.,NaN,transportation and environmental services,NaN,NaN,"330, rue Saint-Vallier Est, Suite 340|Québec, ...",330 Rue Saint-Vallier Est,...,NaN,NaN,2018-09-05,http://www.sedar.com/DisplayProfile.do?lang=EN...,SEDAR,h2o innovation,330 rue saint vallier est suite 340 quebec que...,42EO38IIHZ,1.0,1.0
1,1,NaN,NaN,H3O Holding Corp.,NaN,consumer products,breweries and beverages,NaN,"P.O. Box 482|Beckley, West Virginia|25802-0482",NaN,...,NaN,NaN,2018-09-05,http://www.sedar.com/DisplayProfile.do?lang=EN...,SEDAR,h3o holding,p o box 482 beckley west virginia 25802 0482,IIL07JM0YC,1.0,1.0


In [5]:
batch_data.head(2)

,data_of_signing,country_of_transaction,principal_counterparty,edc_product,transaction_description,range,canadaian_company,retrieved_at,CANADIAN_COMP_ID,address
0,2018-06-06,United States,Royal Bank of Canada (HC Commercial Inc),Guarantee,Support for Canadian Direct Investment Abroad,< 1 mln.,HC Commercial Inc,2018-07-25,PK1LY18EA5,NaN
1,2018-05-23,China,National Bank of Canada (HWF Capital Inc.),Guarantee,Support for Canadian Direct Investment Abroad,< 1 mln.,HWF Capital Inc.,2018-07-25,MF96V4YFO1,NaN


#### 1- clean and preprocess data files based on config file

In [7]:
!sudo python prep_columns_for_er.py

#### sample of intermediate cleaned data for resolver process

In [8]:
cleaned_master_data = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/data_files/cbd_master_deduped_sub_cleaned.csv',
                                  dtype = object)
cleaned_batch_data = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/data_files/sample_edc_sub_cleaned.csv',
                                 dtype = object)

In [10]:
cleaned_master_data.head(5)

,business_name,cleaned_business_name
0,H2O INNOVATION INC.,h2o innovation
1,H3O Holding Corp.,h3o holding
2,Habitations Meridiem Beloeil Inc.,habitations meridiem beloeil
3,HADDINGTON INTERNATIONAL RESOURCES LIMITED,haddington resources
4,Hadrian Energy Corp.,hadrian energy


In [11]:
cleaned_batch_data.head(5)

,canadaian_company,cleaned_canadaian_company
0,HC Commercial Inc,hc commercial
1,HWF Capital Inc.,hwf capital
2,Hitek Distribution Inc.,hitek distribution
3,Hunter Marine Foods Inc.,hunter marine foods
4,Hollywood Alliance Canada Inc.,hollywood alliance canada


#### 2- Run resolvER (link) process with the same config file

In [14]:
!sudo python resolver_flow.py

Loading the embedding model.
Embedding model loaded --- 18.5117328167 seconds ---
--- 0.124500989914 seconds (Values were tokenized) ---
--- 8.40008616447 seconds --- (All values were encoded)
--- 1.39906811714 seconds --- (Weighted average embeddings computed)
--- 1.12330508232 seconds (Vectors were added to the index) ---
--- 14.9155509472 seconds (Index tree was built) ---
Index tree have been saved.
Two datasets values have been provided -- switched to link mode --
('number of iterations: ', 1)
New process started
New process finished
All processes for 0 are done.
All processes for all iterations are done.
Time taken
2.19225120544
Resolving is done... --- 2.19836401939 seconds ---
Saving results ... 
Results have been saved.


#### intermediate results: 
#### - for each row in cleaned batch, if there is a match, the index number of the match in the master is returned

In [15]:
matches = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/intermediate_files/match_res_sample_cbd_edc.csv')
scores = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/intermediate_files/match_scores_sample_cbd_edc.csv')
res_frame = pd.concat([matches, scores], axis = 1)

In [17]:
res_frame.head(10) # clusts here refers to the index ID in master data

,clusts,0
0,10367,1.0
1,-1,-1.0
2,14559,1.0
3,-1,-1.0
4,-1,-1.0
5,-1,-1.0
6,16062,1.0
7,353,1.0
8,-1,-1.0
9,-1,-1.0


#### building back the original batch data frame with matches from master frame

In [18]:
!sudo python link_datasetst.py

sys:1: DtypeWarning: Columns (2,4,7,18,19,20,21,22,23,24,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.


#### OUTPUT:

In [19]:
fin_res = pd.read_csv('/home/mehrsa/data_works/RBC_results/ER_tests/data_files/sample_edc_with_matches_new.csv')

##### sample: rows that had a match:

In [21]:
fin_res[fin_res.business_name.notnull()]

,data_of_signing,country_of_transaction,principal_counterparty,edc_product,transaction_description,range,canadaian_company,retrieved_at,CANADIAN_COMP_ID,address,business_name,0
0,2018-06-06,United States,Royal Bank of Canada (HC Commercial Inc),Guarantee,Support for Canadian Direct Investment Abroad,< 1 mln.,HC Commercial Inc,2018-07-25,PK1LY18EA5,NaN,HC COMMERCIAL INC.,1.00
2,2018-05-22,Morocco,Royal Bank of Canada (Hitek Distribution Inc.),Guarantee,Support for Canadian Direct Investment Abroad,< 1 mln.,Hitek Distribution Inc.,2018-07-25,ULG4PGXUJJ,NaN,HITEK DISTRIBUTION INC.,1.00
6,2018-04-16,Canada,Bank of Montreal (Hardy Filtration Inc.),Guarantee,Sale of various Canadian goods and/or services,< 1 mln.,Hardy Filtration Inc.,2018-07-25,ZW393PPDI6,NaN,Hardy Filtration Inc.,1.00
7,2018-04-16,Dominican Republic,Elevate Export Finance Corp. (Packaging Soluti...,Guarantee,Sale of various Canadian goods and/or services,< 1 mln.,Husky Injection Molding Systems Ltd.,2018-07-25,RJW3GW86WL,NaN,Husky Injection Molding Systems Ltd.,1.00
10,2018-02-28,United States,HSBC Bank Canada (Hatley-Little Blue House Inc.),Guarantee,Sale of various Canadian goods and/or services,1-5 mln.,Hatley-Little Blue House Inc.,2018-07-25,R041BZC5KK,NaN,Hatley Little Blue House Inc.,1.00
12,2018-02-06,United States,The Toronto-Dominion Bank (High Tide Fisheries...,Guarantee,Sale of various Canadian goods and/or services,< 1 mln.,High Tide Fisheries Ltd.,2018-07-25,XLVHMK6RWZ,NaN,HIGH TIDE FISHERIES LTD.,1.00
13,2018-02-05,Saudi Arabia,Royal Bank of Canada (Human Resource Systems G...,Guarantee,Sale of various Canadian goods and/or services,< 1 mln.,Human Resource Systems Group Ltd.,2018-07-25,ZT9HSTI9N5,NaN,Human Resource Systems Group Limited,1.00
14,2018-01-09,Ireland,HSBC Bank Canada (Hartley & Marks Publishers I...,Guarantee,Sale of various Canadian goods and/or services,5-15 mln.,Hartley & Marks Publishers Inc.,2018-07-25,DVW555LTNT,NaN,Hartley and Marks Publishers Inc,0.98
15,2017-12-22,United States,HSBC Bank Canada (Hartley & Marks Publishers I...,Guarantee,Sale of various Canadian goods and/or services,< 1 mln.,Hartley & Marks Publishers Inc.,2018-07-25,DVW555LTNT,NaN,Hartley and Marks Publishers Inc,0.98
16,2017-12-22,Honduras,"Fibras del Caribe, S.A. de C.V.",Financing,Sale of machinery and/or equipment,1-5 mln.,Husky Injection Molding Systems Ltd.,2018-07-25,RJW3GW86WL,NaN,Husky Injection Molding Systems Ltd.,1.00
